# Modelling the Spread

In [77]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

# Load the data

In [78]:
df_analyse=pd.read_csv(r'C:\Users\Aravind\Desktop\Data Science\My Notebook\ds_covid-19\data\processed\COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,India,Italy,Germany
838,2022-05-09,0.031237,0.282392,0.305224
839,2022-05-10,0.031240,0.283335,0.306390
840,2022-05-11,0.031242,0.284052,0.307458
841,2022-05-12,0.031244,0.284715,0.308287
842,2022-05-13,0.031246,0.285370,0.309030


In [79]:
df_analyse['India'] = df_analyse['India'].multiply(1380000000)
df_analyse['Italy'] = df_analyse['Italy'].multiply(59550000)
df_analyse['Germany'] = df_analyse['Germany'].multiply(83240000)

In [80]:
df_analyse_1=df_analyse.dropna()

In [81]:
df_analyse_1.sort_values('date',ascending=True).tail()

,date,India,Italy,Germany
838,2022-05-09,43107689.0,16816419.0,25406868.0
839,2022-05-10,43110586.0,16872618.0,25503878.0
840,2022-05-11,43113413.0,16915301.0,25592839.0
841,2022-05-12,43116254.0,16954784.0,25661838.0
842,2022-05-13,43119112.0,16993813.0,25723697.0


# Helper Functions

In [82]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [83]:
quick_plot(df_analyse_1.date,
           df_analyse_1.iloc[:,1:],
           y_scale='linear',
           slider=True)

In [84]:
threshold=100

In [85]:
compare_list=[]
for pos,country in enumerate(df_analyse_1.columns[1:]):
    compare_list.append(np.array(df_analyse_1[country][df_analyse_1[country]>threshold]))

In [86]:
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

In [87]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [88]:
pd_sync_timelines.head()

,India,Italy,Germany,date
0,102.0,155.0,117.0,0
1,113.0,229.0,150.0,1
2,119.0,322.0,188.0,2
3,142.0,453.0,240.0,3
4,156.0,655.0,349.0,4


In [89]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

In [90]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [91]:
max_days=34

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(10),1),
    'doubling every two days':doubling_rate(100,np.arange(20),2),
    'doubling every 4 days':doubling_rate(100,np.arange(20),4),
    'doubling every 10 days':doubling_rate(100,np.arange(20),10),
}

In [92]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [93]:
pd_sync_timelines_w_slope.head(200)

,doubling every two days,doubling every 4 days,doubling every 10 days,India,Italy,Germany,date
0,100.000000,100.000000,100.000000,102.0,155.0,117.0,0
1,141.421356,118.920712,107.177346,113.0,229.0,150.0,1
2,200.000000,141.421356,114.869835,119.0,322.0,188.0,2
3,282.842712,168.179283,123.114441,142.0,453.0,240.0,3
4,400.000000,200.000000,131.950791,156.0,655.0,349.0,4
5,565.685425,237.841423,141.421356,194.0,888.0,534.0,5
6,800.000000,282.842712,151.571657,244.0,1128.0,684.0,6
7,1131.370850,336.358566,162.450479,330.0,1694.0,847.0,7
8,1600.000000,400.000000,174.110113,396.0,2036.0,1112.0,8
9,2262.741700,475.682846,186.606598,499.0,2502.0,1296.0,9


In [94]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:6],
           y_scale='log',
           slider=True)

In [95]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

# Scikit Linear Regression

# Prediction for Germany

In [96]:
from sklearn import linear_model
reg_germany = linear_model.LinearRegression(fit_intercept=False)

In [97]:
l_vec=len(df_analyse_1['Germany'])
X=np.arange(l_vec-5).reshape(-1, 1)
y=np.log(np.array(df_analyse_1['Germany'][5:]))

In [98]:
reg_germany.fit(X,y)

LinearRegression(fit_intercept=False)

In [99]:
X_hat=np.arange(l_vec).reshape(-1, 1)
Y_hat=reg_germany.predict(X_hat)

In [100]:
LR_inspect=df_analyse_1[['date','Germany']].copy()

In [101]:
LR_inspect['prediction']=np.exp(Y_hat)

In [102]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

# Piecewise Linear Regression

In [103]:
from sklearn import linear_model
from scipy import signal
reg = linear_model.LinearRegression(fit_intercept=True)

In [104]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
country_list=df_analyse.columns[1:]
df_analyse['India'] = df_analyse['India'].multiply(1380000000)
df_analyse['Italy'] = df_analyse['Italy'].multiply(59550000)
df_analyse['Germany'] = df_analyse['Germany'].multiply(83240000)

In [105]:
df_analyse_2=df_analyse

In [106]:
df_analyse_2.head()

,date,India,Italy,Germany
0,2020-01-22,0.0,0.0,0.0
1,2020-01-23,0.0,0.0,0.0
2,2020-01-24,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0
4,2020-01-26,0.0,0.0,0.0


In [107]:
for each in country_list:
    df_analyse_2[each+'_filter']=signal.savgol_filter(df_analyse_2[each],
                                                     5, # window size used for filtering
                           1) # order of fitted polynomial

In [108]:
filter_cols=['Italy_filter','India_filter', 'Germany_filter']

In [109]:
df_analyse_2.tail()

,date,India,Italy,Germany,India_filter,Italy_filter,Germany_filter
838,2022-05-09,43107689.0,16816419.0,25406868.0,43107856.6,16834221.8,25419767.0
839,2022-05-10,43110586.0,16872618.0,25503878.0,43110668.6,16871624.0,25492944.6
840,2022-05-11,43113413.0,16915301.0,25592839.0,43113410.8,16910587.0,25577824.0
841,2022-05-12,43116254.0,16954784.0,25661838.0,43116262.2,16954282.4,25656985.8
842,2022-05-13,43119112.0,16993813.0,25723697.0,43119113.6,16997977.8,25736147.6


In [110]:
start_pos=5
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2[filter_cols].iloc[start_pos:,:],
           y_scale='log',
           slider=True)


In [111]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [112]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [113]:
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse_2[country+'_DR']=df_analyse_2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [114]:
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse_2[country+'_DR']=df_analyse_2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [115]:
df_analyse_2['Germany_DR_math']=df_analyse_2['Germany'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)

In [116]:
df_analyse_2.columns

Index(['date', 'India', 'Italy', 'Germany', 'India_filter', 'Italy_filter',
       'Germany_filter', 'India_DR', 'Italy_DR', 'Germany_DR',
       'Italy_filter_DR', 'India_filter_DR', 'Germany_filter_DR',
       'Germany_DR_math'],
      dtype='object')

In [117]:
start_pos=40
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2.iloc[start_pos:,[7,8,9,10,11,12,13]], #
           y_scale='linear',
           slider=True)

In [118]:
start_pos=40
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2.iloc[start_pos:,[7,8,9,10,11,12,13]],
           y_scale='linear',
           slider=True)